In [2]:
!pip install ultralytics

import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
import math
from math import dist
import time


class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0


    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            x, y, w, h = rect
            cx = (x + x + w) // 2
            cy = (y + y + h) // 2

            # Find out if that object was detected already
            same_object_detected = False
            for id, pt in self.center_points.items():
                dist = math.hypot(cx - pt[0], cy - pt[1])

                if dist < 35:
                    self.center_points[id] = (cx, cy)
#                    print(self.center_points)
                    objects_bbs_ids.append([x, y, w, h, id])
                    same_object_detected = True
                    break

            # New object is detected we assign the ID to that object
            if same_object_detected is False:
                self.center_points[self.id_count] = (cx, cy)
                objects_bbs_ids.append([x, y, w, h, self.id_count])
                self.id_count += 1

        # Clean the dictionary by center points to remove IDS not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids


model=YOLO('yolov8s.pt')



def RGB(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE :  
        colorsBGR = [x, y]
        print(colorsBGR)
        

cv2.namedWindow('RGB')
cv2.setMouseCallback('RGB', RGB)

cap=cv2.VideoCapture('veh2.mp4')


my_file = open("coco.txt", "r")
data = my_file.read()
class_list = data.split("\n") 
#print(class_list)

count=0

tracker=Tracker()

cy1=322
cy2=368

offset=6

vh_down={}
counter=[]


vh_up={}
counter1=[]

while True:    
    ret,frame = cap.read()
    if not ret:
        break
    count += 1
    if count % 3 != 0:
        continue
    frame=cv2.resize(frame,(1020,500))
   

    results=model.predict(frame)
 #   print(results)
    a=results[0].boxes.data
    px=pd.DataFrame(a).astype("float")
#    print(px)
    list=[]
             
    for index,row in px.iterrows():
#        print(row)
 
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' in c:
            list.append([x1,y1,x2,y2])
    bbox_id=tracker.update(list)
    for bbox in bbox_id:
        x3,y3,x4,y4,id=bbox
        cx=int(x3+x4)//2
        cy=int(y3+y4)//2
        
        cv2.rectangle(frame,(x3,y3),(x4,y4),(0,0,255),2)
        


        if cy1<(cy+offset) and cy1 > (cy-offset):
            
            vh_down[id]=time.time()
        if id in vh_down:
            if cy2 < (cy + offset) and cy2 > (cy - offset):
                
                
                if counter.count(id) == 0:
                    elapsed_time = time.time() - vh_down[id]
                    counter.append(id)
                    distance = 10  # meters
                    a_speed_ms = distance / elapsed_time
                    a_speed_kh = a_speed_ms * 3.6
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

                
        #####going UP#####     
        if cy2<(cy+offset) and cy2 > (cy-offset):
            
            vh_up[id]=time.time()
        if id in vh_up:
            if cy1 < (cy + offset) and cy1 > (cy - offset):
                  # Define elapsed_time here
                if counter1.count(id) == 0:
                    elapsed_time = time.time() - vh_up[id]
                    print(elapsed_time)
                    counter1.append(id)
                    distance1 = 10  # meters
                    a_speed_ms1 = distance / elapsed_time
                    a_speed_kh1 = a_speed_ms * 3.6
                    cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)
                    cv2.putText(frame, str(id), (x3, y3), cv2.FONT_HERSHEY_COMPLEX, 0.6, (255, 255, 255), 1)
                    cv2.putText(frame, str(int(a_speed_kh)) + 'Km/h', (x4, y4), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

           

    cv2.line(frame,(274,cy1),(814,cy1),(255,255,255),1)

    cv2.putText(frame,('L1'),(277,320),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)


    cv2.line(frame,(177,cy2),(927,cy2),(255,255,255),1)
 
    cv2.putText(frame,('L2'),(182,367),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    d=(len(counter))
    u=(len(counter1))
    cv2.putText(frame,('goingdown:-')+str(d),(60,90),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)

    cv2.putText(frame,('goingup:-')+str(u),(60,130),cv2.FONT_HERSHEY_COMPLEX,0.8,(0,255,255),2)
    cv2.imshow("RGB", frame)
    if cv2.waitKey(1)&0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()




0: 320x640 1 person, 3 cars, 1 fire hydrant, 153.2ms
Speed: 2.5ms preprocess, 153.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 139.9ms
Speed: 3.2ms preprocess, 139.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 126.9ms
Speed: 2.0ms preprocess, 126.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 143.4ms
Speed: 2.1ms preprocess, 143.4ms inference, 1.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 3 cars, 1 fire hydrant, 126.0ms
Speed: 2.0ms preprocess, 126.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 cars, 1 fire hydrant, 129.3ms
Speed: 1.0ms preprocess, 129.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 persons, 3 cars, 1 fire hydrant, 132.9ms
Speed: 2.0ms preprocess, 132.9ms inference,

[667, 17]
[667, 14]
[667, 11]



0: 320x640 1 person, 2 cars, 114.2ms
Speed: 2.0ms preprocess, 114.2ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 1 car, 126.4ms
Speed: 1.0ms preprocess, 126.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 3 persons, 2 cars, 127.6ms
Speed: 2.0ms preprocess, 127.6ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)



[613, 15]
[612, 15]
[607, 15]
[606, 15]


0: 320x640 1 person, 2 cars, 125.1ms
Speed: 2.0ms preprocess, 125.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 129.0ms
Speed: 1.0ms preprocess, 129.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 131.7ms
Speed: 1.0ms preprocess, 131.7ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 129.5ms
Speed: 1.0ms preprocess, 129.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 127.0ms
Speed: 1.0ms preprocess, 127.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 2 cars, 134.4ms
Speed: 2.4ms preprocess, 134.4ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 133.9ms
Speed: 2.0ms preprocess, 133.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 2 cars, 124.8ms
Speed: 1.0ms preprocess, 124.8m

...

1.782012939453125


0: 320x640 6 cars, 207.9ms
Speed: 3.6ms preprocess, 207.9ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 1 traffic light, 174.9ms
Speed: 3.3ms preprocess, 174.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 173.5ms
Speed: 2.0ms preprocess, 173.5ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 136.7ms
Speed: 2.6ms preprocess, 136.7ms inference, 2.1ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 147.7ms
Speed: 1.5ms preprocess, 147.7ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 156.6ms
Speed: 2.0ms preprocess, 156.6ms inference, 1.7ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 218.9ms
Speed: 2.9ms preprocess, 218.9ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 240.4ms
Speed: 2.0ms preprocess, 240.4ms inference, 3.0ms postprocess per image at

1.5779190063476562


0: 320x640 7 cars, 174.1ms
Speed: 2.0ms preprocess, 174.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 178.2ms
Speed: 2.0ms preprocess, 178.2ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 160.1ms
Speed: 1.4ms preprocess, 160.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 181.0ms
Speed: 2.0ms preprocess, 181.0ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 222.1ms
Speed: 1.9ms preprocess, 222.1ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 6 cars, 184.3ms
Speed: 2.5ms preprocess, 184.3ms inference, 1.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 7 cars, 223.1ms
Speed: 3.0ms preprocess, 223.1ms inference, 2.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 5 cars, 203.6ms
Speed: 3.0ms preprocess, 203.6ms inference, 3.0ms postprocess per image at shape (1, 3, 320

KeyboardInterrupt: 